In [1]:
!pip install "transformers==4.35" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.42.0" "trl==0.4.7" "safetensors>=0.3.1" "tiktoken"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from random import randrange
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
torch_dtype=torch.float16

# Load dataset from local

In [4]:
df = pd.read_csv("QA_Data.csv")

In [5]:
df.head()

,Unnamed: 0,context,question,answers
0,0,\nAmenirdis I (throne name: Hatneferumut) was ...,Who was Amenirdis I?,Amenirdis I was a God's Wife of Amun during th...
1,1,\nAmenirdis I (throne name: Hatneferumut) was ...,Who was Amenhotep III?,"Amenhotep III, also known as Amenhotep the Mag..."
2,2,\nAmenirdis I (throne name: Hatneferumut) was ...,Who was Khafre?,"Khafre, also known as Khephren or Chephren, wa..."
3,3,\nAmenirdis I (throne name: Hatneferumut) was ...,Who was Amenemhat III?,"Amenemhat III, also known as Amenemhet III, wa..."
4,4,\nAmenirdis I (throne name: Hatneferumut) was ...,What was discovered by Professor Maspero in Th...,A coffin and mummy of a gazelle from the 21st ...


In [6]:
# Concatenate 'Question' and 'Answer' columns into a new column 'text'
df['text'] = 'Question:\n' + df['question'] + '\n\nAnswer:\n' + df['answers']

In [7]:
#drop columns other than 'text'
df.drop(columns=['question','answers','context','Unnamed: 0'], axis=1, inplace=True)

In [8]:
#convert to Huggingface Datasets format
train = Dataset.from_pandas(df)

In [9]:
train

Dataset({
    features: ['text'],
    num_rows: 383
})

# Fine-Tuning

In [10]:
model_id = "genaitraining/llama-2-7b-domain-tuned"

In [11]:
# Get the type
compute_dtype = getattr(torch, "float16")

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype
)

In [12]:
%%time
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

CPU times: user 190 ms, sys: 30.3 ms, total: 220 ms
Wall time: 398 ms


In [13]:
%%time
# Load the pretrained model
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 6.39 s, sys: 6.83 s, total: 13.2 s
Wall time: 58.7 s


In [14]:
# LoRA config based on QLoRA paper
peft_config = LoraConfig(
                          lora_alpha=16,
                          lora_dropout=0.1,
                          r=64,
                          bias="none",
                          task_type="CAUSAL_LM"
                        )

In [15]:
# Define the training arguments. For full list of arguments, check
#https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments
args = TrainingArguments(
    output_dir='/content/drive/MyDrive/llama2-7b',
    num_train_epochs=50, # adjust based on the data size
    per_device_train_batch_size=4, # use 4 if you have more GPU RAM
    save_strategy="epoch", #steps
    # evaluation_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    seed=42
)

In [16]:
# Create the trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    # eval_dataset=test,
    dataset_text_field='text',
    peft_config=peft_config,
    max_seq_length=1042,
    tokenizer=tokenizer,
    args=args,
    packing=True,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [17]:
# train
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=250, training_loss=0.3134870300292969, metrics={'train_runtime': 3399.1362, 'train_samples_per_second': 5.634, 'train_steps_per_second': 1.412, 'total_flos': 3.73670051364864e+16, 'train_loss': 0.3134870300292969, 'epoch': 49.05})

In [18]:
# save model in local
trainer.save_model()

Reload the saved model and merge it then we can save the whole model

In [ ]:
%%time
from peft import AutoPeftModelForCausalLM

new_model = AutoPeftModelForCausalLM.from_pretrained(
    '/content/drive/MyDrive/llama2-7b/checkpoint-250',
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 19.9 s, sys: 9.09 s, total: 29 s
Wall time: 1min 22s


In [ ]:
prompt = "Who was Amenhotep III?"


In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
outputs = new_model.generate(input_ids=input_ids,
                         max_new_tokens=200,
                         do_sample=True,
                        top_p=0.9,
                         temperature=0.6)

In [ ]:
result = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

In [ ]:

print(result)

Who was Amenhotep III?

Amenhotep III (also known as Amenhotep the Magnificent) was an Egyptian pharaoh of the 18th dynasty, who ruled Egypt from around 1386 to 1349 BC. He is known for his extensive building projects and military campaigns, as well as his diplomatic efforts to maintain peace with Egypt's neighbors.

Amenhotep III was born around 1386 BC, the son of Tutankhamun and Queen Tiye. He ascended to the throne at around the age of 10, and his early years were marked by the influence of his powerful wet nurse, Kiya. Amenhotep III was known for his love of luxury and his extensive building projects, which included the construction of temples, palaces, and other monumental structures throughout Egypt. He also expanded Egypt's trade networks and encourag
